In [0]:
import torch
import torchvision
from torch import nn
from torch.autograd import Variable
from torch.utils.data import DataLoader
from torchvision import transforms
from torchvision.utils import save_image
from torchvision.datasets import MNIST
import os
import pandas as pd
import numpy as np
import random
random.seed(7)

In [6]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"witcher0709","key":"8da736a9b1e337a294dd73dce2c4d863"}'}

In [0]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!kaggle datasets list

In [8]:
!kaggle competitions download -c LANL-Earthquake-Prediction

  0% 0.00/33.3k [00:00<?, ?B/s]
100% 33.3k/33.3k [00:00<00:00, 14.2MB/s]
 97% 234M/242M [00:01<00:00, 137MB/s]
100% 242M/242M [00:01<00:00, 145MB/s]
100% 2.02G/2.03G [00:17<00:00, 129MB/s]
100% 2.03G/2.03G [00:17<00:00, 123MB/s]


In [9]:
!mkdir data
!mv train.csv.zip data/
!mv test.zip data/
!unzip data/train.csv.zip
!mkdir data/test
!unzip -q data/test.zip
!mv train.csv data/
!mv seg* data/test/

Archive:  data/train.csv.zip
  inflating: train.csv               


In [0]:
df  = pd.read_csv('data/train.csv',nrows=15000000,dtype={'acoustic_data': np.int16, 'time_to_failure': np.float64})
ttf = df['time_to_failure'].values
index_start = np.nonzero(np.diff(ttf) > 0)[0] + 1
index_start = np.insert(index_start, 0, 0)
dict_df={}
for i in range(len(index_start)-1):
  df_tmp=df[index_start[i]:index_start[i+1]]
  dict_df["df"+str(i)]=df_tmp

In [28]:
len(dict_df)

1

In [0]:
df_seg  = pd.read_csv('data/test/seg_00030f.csv',dtype={'acoustic_data': np.int16, 'time_to_failure': np.float64})

In [27]:
def sampler(df_dict):
  k = len(df_dict)
  num=random.random.randint(0,k)
  len_df=len(df_dict['df'+str(num)])
  df_tmp=df_dict['df'+str(num)]
  idx_start=random.randint(0,len_df-150000)
  idx_end=idx_start+150000
  sample_x=df_tmp.iloc[idx_start:idx_end]['acoustic_data'']
  sample_y=df_tmp.iloc[idx_start:idx_end]['time_to_failure']
  sample_x=np.array(sample_x)
  sample_y=np.array(sample_y)
  sample_x.rehape()
  
  
  

IndexError: ignored

In [0]:
img_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

dataset = MNIST('./data', transform=img_transform)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

In [0]:
num_epochs = 100
batch_size = 128
learning_rate = 1e-3

In [0]:
class autoencoder(nn.Module):
    def __init__(self):
        super(autoencoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Conv2d(1, 16, 3, stride=3, padding=1),  # b, 16, 10, 10
            nn.ReLU(True),
            nn.MaxPool2d(2, stride=2),  # b, 16, 5, 5
            nn.Conv2d(16, 8, 3, stride=2, padding=1),  # b, 8, 3, 3
            nn.ReLU(True),
            nn.MaxPool2d(2, stride=1)  # b, 8, 2, 2
        )
        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(8, 16, 3, stride=2),  # b, 16, 5, 5
            nn.ReLU(True),
            nn.ConvTranspose2d(16, 8, 5, stride=3, padding=1),  # b, 8, 15, 15
            nn.ReLU(True),
            nn.ConvTranspose2d(8, 1, 2, stride=2, padding=1),  # b, 1, 28, 28
            nn.Tanh()
        )

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x


In [0]:
model = autoencoder().cuda()
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate,
weight_decay=1e-5)

In [0]:
for epoch in range(num_epochs):
    for data in dataloader:
        img, _ = data
        img = Variable(img).cuda()
        # ===================forward=====================
        output = model(img)
        loss = criterion(output, img)
        # ===================backward====================
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    # ===================log========================
    print('epoch [{}/{}], loss:{:.4f}'
          .format(epoch+1, num_epochs, loss.data[0]))
    if epoch % 10 == 0:
        pic = to_img(output.cpu().data)
save_image(pic, './dc_img/image_{}.png'.format(epoch))

In [0]:

torch.save(model.state_dict(), './conv_autoencoder.pth')